# Exercise 2

# Transform the recorded robot positions into a Self-Organizing Map (SOM)

## Introduction <a id='I'></a>

</br><div style="text-align: justify">In a self-organizing map (SOM), the neurons are placed at the nodes of a lattice that is usually one- or two-dimensional. Higher-dimensional maps are also possible but not as common. The neurons become selectively tuned to various input patterns (stimuli) or classes of input patterns in the course of a competitive learning process [1]. </div>

</br><div style="text-align: justify">The locations of the neurons so tuned (i.e., the winning neurons) become ordered with respect to each other in such a way that a meaningful coordinate system for different input features is created over the lattice [2]. </div>

</br><div style="text-align: justify">A self-organizing map is therefore characterized by the formation of a topographic map of the input patterns in which the spatial locations (i.e., coordinates) of the neurons in the lattice are indicative of intrinsic statistical features contained in the input patterns, hence the name "self-organizing map". So, the principle of topographic map formation may be stated as: </div>

</br><div style="text-align: center">*" The spatial location of an output neuron in a topographic map corresponds to a particular domain or feature of data drawn the input space. "*</div>

</br><div style="text-align: justify">Based on this principle, we can say that since a self-organizing map is inherently non-linear it may be viewed as a non-linerar generalization of principal components analysis [3]. Thus, **the task we are going to solve in this exercise is to reduce the number of recorded robot positions by creating a self-orginizing map which captures the main features of the original dataset.**</div>


</br>**References:**<div style="text-align: justify"> [1] S. Haykin (1998), "Neural Networks. A Comprehensive Foundation", Second Edition, Prentice Hall (1998), Chapter 9, pp. 465-505.</div>
</br><div style="text-align: justify">[2] T. Kohonen (1990) The Self-Organizing Map. Proceedings of the IEEE, 78, pp. 1464-1480.</div>
</br><div style="text-align: justify">[3] H. Ritter, Kohonen T. (1995) The Basic SOM. In: Self-Organizing Maps. Springer Series in Information Sciences, vol 30, pp. 77-130. </div>

--------------------------------------

--------------------------------------
## Getting started: solution of Excercise 1 <a id='S'></a>
</br><div style="text-align: justify"> This exercise aims at generating a self-organizing map based on the recorded robot positions (`robot_positions.csv` from **Exercise 1**). If you haven't completed Exercise 1, you should follow next steps first:</div>

- Join the [Neurorobotics platform](http://148.187.97.48/#/esv-private) and select the __*Templates*__ tab.
- Select the experiment whose title starts with ***"Exercise 2: ..."*** (you can use the filter at the top right-end corner).
- Press the __*Clone*__ button; now you own a copy of the experiment which is visible in __*My Experiments*__ tab.


- Open the <span style="background-color: #f6d351">Week 1 - Exercise 1 (Jupyter Notebook)</span> in your Collab's space.
- Move to the part __"C. Record the robot positions"__.
- Execute __"C. Record the robot positions"__ using __Exersice_<font color=red>2_i</font>__ from the list of NRP experiments you will obtain.

**Remark:** Exercise 2's experiment contains the solution of Exercise 1.

--------------------------------------------

--------------------------------------------

### Content:
- <font size="4">**[A. Installation of utils and download of <span style="background-color: #f6d351">Storage</span> files](#A)**</font></br>
>- <font size="3">[A.1. Install SOM utils](#A1)</font></br>
>- <font size="3">[A.2. Download csv input file from this Collab's <span style="background-color: #f6d351">Storage</span>](#A2)</font></br>
- <font size="4">**[B. Complete the SOM implementation](#B)**</font></br>
>- <font size="3">[B.1. SOM description](#B1)</font></br>
>- <font size="3">[B.2. The SOM class (TO DO) ](#B2)</font></br>
- <font size="4">**[C. SOM simulation](#C)**</font></br>
>- <font size="3">[C.1. Perform SOM training ](#C1)</font></br>
>- <font size="3">[C.2. Upload your SOM solution into this Collab's <span style="background-color: #f6d351">Storage</span>](#B3)</font></br>
>- <font size="3">[C.3. Save the feature map](#C2)</font></br>
- <font size="4">**[D. Result evaluation](#D)**</font></br>
- <font size="4">**[E. Submission and grading ](#E)**</font></br>

---------------------------------------------------

---------------------------------------------------
## A. Installation of SOM utils and download of <span style="background-color: #f6d351">Storage</span> files<a id='A'></a>

### A.1. Install SOM utils <a id='A1'></a>
Run the cell below to install SOM utils.

In [ ]:
# Install SOM utils
from IPython.display import clear_output
! pip uninstall epflx_robox_nrp_utils -y
! pip install --user --force-reinstall git+https://github.com/HBPNeurorobotics/EPFLx-RoboX-Neurorobotics-utils#egg=epflx_robox_nrp_utils
! pip install --upgrade "hbp-service-client==1.1.1"
clear_output()

### A.2. Download the csv input file from your Collab <span style="background-color: #f6d351">Storage</span> <a id='A2'></a>
Run the two cells below in order to download the input data.

In [3]:
# Download a selected file from your Collab Storage
def download_from_collab_storage(filename, default):
    print('Downloading...')
    clients = get_hbp_service_client()
    collab_path = get_collab_storage_path()
    if not filename:
        filename = default
    if not filename: 
        raise ValueError('Missing filename argument')
    from os import path
    try:
        clients.storage.download_file(path.join(collab_path, filename), './%(filename)s' % { 'filename': filename})
        print('The file \'%(filename)s\' has been successfully downloaded. It is located here: ./%(filename)s' % { 'filename': filename})
    except Exception:
        print('The file \'%(filename)s\' couldn\'t be found in your storage.' % { 'filename': filename})
        
def display_file_widget(default_filename):
    import ipywidgets as widgets
    from IPython.display import display
    style = {'description_width': 'initial'}
    input_file_widget = widgets.Text(
        description='Filename', 
        placeholder='%(default_filename)s (default)' % {'default_filename': default_filename},
        style=style,
        layout=widgets.Layout(width='50%')
    )
    display(input_file_widget)
    download_button = widgets.Button(description="Download")
    display(download_button)
    def button_callback(b):
        return download_from_collab_storage(filename=input_file_widget.value, default=default_filename)
    download_button.on_click(button_callback)

In [2]:
display_file_widget('robot_positions.csv')

Text(value=u'', description=u'Filename', layout=Layout(width=u'50%'), placeholder=u'robot_positions.csv (defau…

Button(description=u'Download', style=ButtonStyle())

### A.3 Utility function to save files into your Collab <span style="background-color: #f6d351">Storage</span>

In [ ]:
# Utility function used to save files into your Collab storage

"""
Save a file to the Collab storage

:param filepath: relative path to the file
:param filetype: type of the file, e.g., 'text/csv' or 'text/x-python'
:param remove: remove the file of the Jupyter notebook space if True 
"""
def save_to_collab_storage(filepath, filetype, remove=False):
    print('Saving ...')
    import os
    from os import path
    clients = get_hbp_service_client()
    collab_path = get_collab_storage_path()
    filename = path.basename(filepath)
    if os.path.isfile(filename):
        # remove the previous version, if it exists
        filepath = os.path.join(collab_path, filename)
        if clients.storage.exists(filepath): 
            clients.storage.delete(filepath)
        pydata = clients.storage.upload_file(filename, filepath, filetype)
        print("The file %(filename)s has been successfully saved to %(filepath)s" % {"filename": filename, "filepath": filepath})
        # remove 
        if remove:
            os.remove(filename)
    else:
        print("Error: the file %(filename)s couldn't be found." % {"filename": filename})

---------------------------------------------------

---------------------------------------------------
## B. Complete the SOM implementation <a id='B'></a>

### B.1. Description of the SOM class <a id='B1'></a>
****

</br><div style="text-align: justify">**Your task** is to implement the SOM algorithm and to apply it to robot positions data set obtained in Exercise 1 (`robot_positions.csv`). The template of the SOM class is located in part [B.2. Complete SOM function (TO DO)](#B2). Once you have filled in all the gaps, you will run the algorithm on the aforementioned data set. </div>

</br><div style="text-align: justify">The **SOM class template** consists of two parts: **1) `Self-Organizing Map initialization`** and **2) `TO DO`**. The first part allows you to run, to initialize and to visualize the SOM ordering and converging processes via **additional functions** (`SOM_additional`) which are already implemented. You shouldn't change these functions. As for the <font color=red>**"TO DO" part**</font>, this is  <font color=red>**your assignement**</font>. </div>

</br><div style="text-align: justify"> The script in [B.2. Complete SOM function (TO DO)](#B2) represents the SOM algorithm which is summarized below.</div>

****

#### SOM algorithm <a id='SOM'></a>
<font color=green>**while**</font> (**`N_trials`** are not done):<br>
> <font color='#2B8973'>*# TRIAL: __run_trial()__*</font><br>
> shuffle the input dataset: **`shuffle_data()`**<br>
> <font color=green>**while**</font> (not all points have been considered):<br>
>> *<font color='#2B8973'># EPISODE: __run_episode()__</font>*<br>
>> - take the next data point according to a prescribed order<br>
>> - calculate the **`squared_distance()`** from each node to the selected point<br>
>> - determine the winner based on the **`squared_distance()`**: **`get_winner()`**<br>
>> - calculate the **`neighborhood_function()`**<br>
>> - update the SOM-lattice weights with **`update_lattice()`**<br>

****


#### SOM formulas (S. Haykin, "Neural Networks. A Comprehensive Foundation", 1998, Chapter 9, pp. 465--505)
- Squared distance between two neurons or nodes of the SOM lattice (the lattice coordinates are integer coordinates)
\begin{equation} \tag{9.5}
d_{j,i}^2 = \Vert \mathbf{r}_i - \mathbf{r}_j \Vert^2
\end{equation}

- Exponential decay of the size of the winner's topological neighborhood
\begin{equation} \tag{9.6}
\sigma(n) = \sigma_0 \exp\left(-\frac{n}{\tau_1}\right), \, n = 0, 1, 2, \dots 
\end{equation}

- The Gaussian neighborhood function
\begin{equation} \tag{9.7}
h_{j, i(\mathbf{x})}(n) = \exp\left(-\frac{d_{j,i}^2}{2\sigma(n)^2}\right), \, n = 0, 1, 2, \dots 
\end{equation}
where $i(\mathbf{x})$ is the index of the winner corresponding to the input point $\mathbf{x}$.

- The update rule of the synaptic weight vectors
\begin{equation} \tag{9.13}
\mathbf{w}_j(n + 1) = \mathbf{w}_j(n) + \eta(n)h_{j,i}(n)(\mathbf{x} - \mathbf{w}_j(n)), \, n = 0, 1, 2, \dots 
\end{equation}
where $\eta(n)$ is the *learning rate* defined in the next equation.

- Exponential decay of the learning rate
\begin{equation} \tag{9.14}
\eta(n) = \eta_0 \exp\left(-\frac{n}{\tau_2}\right), \, n = 0, 1, 2, \dots 
\end{equation}

</br><div style="text-align: justify">Let us have a look at the template script in [B.2. The SOM class (TO DO)](#B2). As you can see, the part "TO DO" consists of 4 sections to be completed in their natural order. The simulation will be possible only when all parts have been implemented correctly. We believe that the choosen order helps clarify the implementation process. Now it is time <font color=blue>**TO DO**</font> one part after the other: </div>

**Constant parameters: [[1]](#R1)** <br>

</br><div style="text-align: justify">We start with constant parameters of the SOM algorithms. The first set of parameters is made of **non-editable parameters** used for the visualization of the training process. <font color=blue>**The second set of parameters is to be initialized by yourself**</font>. 

**Remark:** Do not change any prescribed function name. Function names are required to be untouched in the submission step. 
</div> 


</br><div style="text-align: justify">Here is the parameter list:   </div> 
<br>
 - **`self.display`:** the visualization option (`'simulation'` to simulate and show only the final lattice, or `'visualization'` to watch all phases).<br>
 - **`self.csv_file`:** the name of the input file (`robot_positions.csv`).<br>
 - **`self.pos`:** the robot positions extracted from the input file.<br>
 - **`self.lattice`:** the initial SOM lattice, to be randomly initialized.<br>
 - **`self.trial`:** the index of the current trial (the initial value is 0).<br>
<br>
 - <font color=blue>**self.Nn:**</font> SOM lattice size [_Nn_ x _Nn_].<br>
 - <font color=blue>**self.eta_0:**</font> the **initial** value of the learning-rate parameter (Formula 9.14 of [1]). <br>
 - <font color=blue>**self.sigma_0:**</font> the **initial** value of exponential decay parameter (Formula 9.6 of [1]).<br>
 - <font color=blue>**self.tau_2:**</font> the time constant of the learning-rate parameter function (Formula 9.14 of [1]).<br>
 - <font color=blue>**self.tau_1:**</font> the time constant of the exponential decay parameter function (Formula 9.6 of [1]).<br>
 - <font color=blue>**self.N_trials:**</font> the number of trials during learning.<br>
 - <font color=blue>**self.lattice_limit:**</font> the range for the **initial** weights of the SOM-lattice (`self.lattice`).<br>

**SOM training stages: [^](#SOM)**<br>
 - **`run_trial()`:** run the SOM algorithm on the complete (shuffled) input dataset; <font color=green>**return**</font> __`nothing`__.<br>
 - **`run_episode()`:** handle one input vector (i.e., one robot pose); <font color=green>**return**</font> __`nothing`__.<br>

__Additional functions: [[2]](#R2)__<br>
 - **`shuffle_data()`:** shuffle the input dataset (i.e., robot positions); <font color=green>**return**</font> __`array`__ of point index sequence.<br>
 - **`squared_distance()`:** the square of the Euclidean distance between a data point and all neuron weights; <font color=green>**return**</font> __`array`__ the distances to all neurons of the SOM.<br>
 - **`get_winner()`:** the winning neuron; <font color=green>**return**</font> coordinates **`[x,y]`** of the winning neuron.<br>
 - **`neighborhood_function()`:** Gaussian neighborhood function (Formula 9.7 of [1]); <font color=green>**return**</font> __`matrix`__ of neuron neighborhood factors according to the current winning neuron.<br>
 - **`update_lattice()`:** update the SOM-lattice according to the SOM learning rule (Formula 9.13 of [1]); <font color=green>**return**</font> __`nothing`__.<br>

__Variable parameters: [[2]](#R2)__<br>
 - **`eta()`:** the learning-rate parameter (Formula 9.14 of [1]); <font color=green>**return**</font> updated __`eta`__ value.<br>
 - **`sigma()`:** the exponential decay parameter (Formula 9.6 of [1]); <font color=green>**return**</font> updated __`sigma`__ value.<br>
<br>

<a id='R1'></a> [1] Once you have implemented all functions, fine tune the __constant parameters__ in order to get a satisfying output map. <br>
<a id='R2'></a> [2] For complete SOM definitions and formulas, see **S. Haykin (1998), "Neural Networks. A Comprehensive Foundation"** (Chapter 9, pp. 465-505). 

****

The **result** of the main function **`run_som()`** is the self-organized map, also called feature map. This map is saved into the file  `lattice.csv` (visualized as red lattice in the forthcoming cells). You can upload this file into your Collab <span style="background-color: #f6d351">Storage</span> via the command [C.2. Upload SOM training result into Collab's storage](#C2). <br>
****

<font color=red>**IMPORTANT:**</font>
<br>
</br><div style="text-align: justify"> **1)** Be aware that the grading of your SOM class implementation will be performed <font color=red>**only with an SOM lattice of size 8x8**</font>. You can use other sizes of SOM to see how it works. For example, we will ask you to use a bigger lattice for Exercise 3 in the reinforcement learning implementation.</div>
</br><div style="text-align: justify">**2)** Keep in mind that the grading process has a timeout. <font color=red>**This timeout is 240 seconds**</font>. If the simulation time exceeds this timeout, the submission will fail.</div>
</br><div style="text-align: justify">**3)** Once you are ready to submit, move to [B.3. Upload SOM function into the storage](#B3) and save your solution into Colllab's <span style="background-color: #f6d351">Storage</span>. </div>
</br><div style="text-align: justify">**4)** Then **[submit](#Su)** your SOM class implementation for grading.</div>

****
****

### B.2. The SOM class (TO DO) <a id='B2'></a>

In [ ]:
#%%writefile SOM_solution.py 

# SOM class
import numpy as np
import random
import time
import math

from IPython import display
from ipywidgets import IntProgress


class SOM():

	#############################################################################################################
	#                                     Self-Organizing Map initialization                                    #
	#############################################################################################################
    
	# function to initialize 'class SOM'
	def __init__(self, display='simulation', csv_file='robot_positions.csv'):
		from epflx_robox_nrp_utils.SOM.SOM_additional import SOM_additional
		self.somad = SOM_additional()
		self.display = display
		self.csv_file = csv_file
		self.init_parameters()
        

	# function to run SOM training
	def run_som(self):       
		T,f,self.Nn = self.somad.som_preparation(self.N_trials,self.Nn,self.display)
        
		"""     SOM algorithm     """
		self.lattice = np.random.uniform(self.lattice_limit[0],self.lattice_limit[1],(self.Nn,self.Nn,2))
		self.pos = self.somad.load_data(self.csv_file)
		self.trial = 0
		while(self.trial < self.N_trials):
			self.run_trial()
			self.trial += 1

			siminfo = [self.Nn, self.N_trials, self.trial, self.eta(), self.sigma()]
			self.somad.visualization(siminfo,self.lattice,self.display,f)
		self.somad.display_results(T,self.display)
        
        
	"""======================================================================================================="""
	"""                                                TO DO                                                  """
	"""======================================================================================================="""
        
	##########################################
	"""         Constant parameters        """
	##########################################
        
	def init_parameters(self):
		self.Nn = 8
		self.eta_0 = 0.1
		self.sigma_0 = 1.0        
		self.tau_2 = 1000.0
		self.tau_1 = 100.0
		self.N_trials = 100.0
		self.lattice_limit = [-3.0, 3.0]


	#########################################
	"""        SOM training stages        """
	#########################################

	def run_trial(self):
		mix = self.shuffle_data();
		for i in range(self.pos.shape[0]):
			self.run_episode(mix, i)


	def run_episode(self, mix, i):
		pt = self.pos[mix[i]]
		d = self.squared_distance(pt)
		[lx,ly] = self.get_winner(d)
		h = self.neighborhood_function(lx[0],ly[0])
		self.update_lattice(h,pt)


	##########################################
	"""        Additional functions        """
	##########################################

	def shuffle_data(self):
		mix = np.arange(self.pos.shape[0])
		np.random.shuffle(mix)
		return mix


	def squared_distance(self, pt):
		return np.linalg.norm(self.lattice - pt,axis=2)**2


	def get_winner(self,d):
		return np.where(d == d.min())


	def neighborhood_function(self,lx,ly):
		h = np.zeros((self.Nn,self.Nn))
		for i in range(self.Nn):
			for j in range(self.Nn):
				dist = (i-lx)**2 + (j-ly)**2
				h[i,j] = np.exp(-dist/(2*self.sigma()**2))
		return h


	def update_lattice(self,h,pt):
		for i in range(self.lattice.shape[2]):
			self.lattice[:,:,i] += self.eta()*h*((pt-self.lattice)[:,:,i])
    

	#########################################
	"""        Variable parameters        """
	#########################################

	def eta(self):
		return self.eta_0*np.exp(-self.trial/self.tau_2)


	def sigma(self):
		return self.sigma_0*np.exp(-self.trial/self.tau_1)

----------------------------------

----------------------------------
## C. SOM simulation <a id='C'></a>

### C.1. Perform SOM training <a id='C1'></a>

Once you have implemented [B.2 The SOM class (TO DO)](#B2), you can perform SOM training. You can run and visualize the training process with the following python commands:

```python
som = SOM(mode_name) # instantiate your SOM class (replace mode_name for 'simulation' or 'visualization')
som.run_som() # run simulation
```
`'simulation'` mode doesn't provide with any visualization;<br>
`'visualization'` mode provides with visualization of SOM deployment state after every trial.

In [ ]:
som = SOM('simulation')
som.run_som()

****
### C.2. Upload your SOM solution into your Collab <span style="background-color: #f6d351">Storage</span><a id='B3'></a>
To save your solution file, follow these steps: </br>

- Move to [B.2. SOM function](#B2) and uncomment the first line of the script, that is remove the "#" placed before `%%writefile SOM_solution.py`;
- Run the cell [B.2. SOM function](#B2). You should see `Writing SOM_solution.py` just above; 
- Run the cell below to save your solution file into your Collab storage.

In [ ]:
save_to_collab_storage(filepath='SOM_solution.py', filetype='text/x-python', remove=True)

### C.2. Save the feature map<a id='C2'></a>
The output of the SOM training is the SOM lattice, also called feature map. This lattice is represented with the red color on the visualization figure. 
The lattice data was saved into the file `lattice.csv` which is now contained in this Jupyter notebook space. This commands upload the `lattice.csv` into your Collab <span style="background-color: #f6d351">Storage</span>. 

Run the cell below and then check in your Collab <span style="background-color: #f6d351">Storage</span> if the `lattice.csv` has been saved.

In [ ]:
save_to_collab_storage(filepath='lattice.csv', filetype='text/csv', remove=False)

---------------------------------------------------------------------

---------------------------------------------------------------------
## D. SOM evaluation <a id='D'></a>

Run the next cell to get a score for your implementation. This score is an average distance between data points (blue dots) and the centers of clusters (red dots) represented as a lattice. The goal of SOM training is to minimize this distance as much as possible.

In [ ]:
# Evaluation of the SOM training
from epflx_robox_nrp_utils.SOM.SOM_evaluation import SOM_evaluation
somev = SOM_evaluation()
var, N = somev.run_evaluation()
print "Your score is ", var, "with SOM size (", N, "x", N,")."

## E. Submission and grading

Run the next cell to submit your SOM implementation. Each submission is automatically tested and evaluated. Only the last submission will be taken into account for your grade. <font color=red>**Keep in mind that the grading process has a timeout of 240 seconds.**</font>

In [ ]:
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
print("Retrieving your HBP OIDC token ...")
token = str(oauth.get_token())
print("token retrieved!")
submission_info = {
    'subheader': 'Exercise 2',
    'filepath': 'SOM_solution.py',
    'token': token,
    'collab_path': collab_path,
    'clients_storage': clients.storage
}
from epflx_robox_nrp_utils.submission_manager.submission_widget import display_submission_widget
display_submission_widget(submission_info)